In [1]:
import pandas as pd
from collections import Counter

In [2]:
import os
import random
import numpy as np
import pickle
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)
import seaborn as sns
import matplotlib.pyplot as plt

In [76]:
def normalizeTargets(i,file):
    ratefile =  os.path.join(file,file+"_rate.npy")
    
    nTargets_1 =np.load(ratefile)

    norm = 'zscore'

    if(norm == 'zscore'):
        tar_mean = np.mean(nTargets_1,axis=0)
        tar_sd = np.std(nTargets_1,axis=0)
        nTargets =nTargets_1- tar_mean
        nTargets = np.divide(nTargets,tar_sd,out=np.zeros_like(nTargets),where=tar_sd!=0)

    return nTargets[i]

In [203]:
def de_normalizeTargets(error,file='train'):
    ratefile =  os.path.join(file,file+"_rate.npy")

    nTargets_1 =np.load(ratefile)


    tar_mean = np.mean(nTargets_1,axis=0)
    tar_sd = np.std(nTargets_1,axis=0)
    erro = np.dot(error,tar_sd)+tar_mean


    return erro

In [204]:
def SNP_WINDOW(w,r,i,file):
 
    respectiveNormalizedTargets = [normalizeTargets(i,file)]
    targets = np.array(respectiveNormalizedTargets)
    lis=[]

    for i in w:
        co = dict(Counter(i))
        if len(co.keys())==2:
            lis.append(co[min(co,key=co.get)]/len(i))
    bins = np.linspace(0, 1, 20)

    bin_means,bins = np.histogram(lis, bins, normed=True, density=True)
    bins= bins[1:]
    bin_means= bin_means/bin_means.sum()
    

    
    

    return np.array(bins),np.array(targets),bin_means

In [205]:
def break_window(ha,po,ra,winsize):
    print(len(ha))
    bins = np.linspace(0, len(ha),winsize)

    bin_means,bins = np.histogram(ha, bins)
    print(bin_means)

In [206]:
def __data_generation1(batchTreeIndices,file):

        haps = []
        pos = []
        rate =[]
        for treeIndex in batchTreeIndices:
            Hfilepath = os.path.join(file,str(treeIndex) + "_haps.npy")
            Pfilepath = os.path.join(file, str(treeIndex) +"_pos.npy")
            H = np.load(Hfilepath)
            P = np.load(Pfilepath)

            haps.append(H)
            pos.append(P)

        return haps,pos


            
def __getitem__1(idx,val):
    X,x1= __data_generation1([idx],val)
    return X,x1

In [207]:
Haps,Pos = __getitem__1(0,'train')

In [208]:
_,target,feature =SNP_WINDOW(Haps[0],Pos[0],0,'train')

/tmp/ipykernel_10058/3638402278.py:13: DeprecationWarning: The normed argument is ignored when density is provided. In future passing both will result in an error.
  bin_means,bins = np.histogram(lis, bins, normed=True, density=True)


In [209]:
pd.DataFrame(feature)

,0
0,0.215054
1,0.150538
2,0.118280
3,0.150538
4,0.064516
5,0.053763
6,0.096774
7,0.053763
8,0.086022
9,0.010753


In [210]:
import xgboost as xg
from sklearn.metrics import mean_squared_error as MSE

In [211]:
xgb_r = xg.XGBRegressor(objective ='reg:linear',n_estimators = 10, seed = 123)

In [212]:
feature_= []
target_=[]
for i in range(2):
    Haps,Pos = __getitem__1(i,'train')
    _,target,feature =SNP_WINDOW(Haps[0],Pos[0],i,'train')
    feature_.append(feature)
    target_.append(target[0])

/tmp/ipykernel_10058/3638402278.py:13: DeprecationWarning: The normed argument is ignored when density is provided. In future passing both will result in an error.
  bin_means,bins = np.histogram(lis, bins, normed=True, density=True)


In [213]:
xgb_r.fit(pd.DataFrame(np.stack(feature_,axis=1)).T, target_)
#pred = xgb_r.predict(np.vstack(feature))

[14:42:45] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=10, n_jobs=None, num_parallel_tree=None,
             objective='reg:linear', predictor=None, ...)

In [214]:
feature_= []
target_=[]
Haps,Pos = __getitem__1(2,'train')
_,target,feature =SNP_WINDOW(Haps[0],Pos[0],2,'train')
feature_.append(feature)
target_.append(target[0])

/tmp/ipykernel_10058/3638402278.py:13: DeprecationWarning: The normed argument is ignored when density is provided. In future passing both will result in an error.
  bin_means,bins = np.histogram(lis, bins, normed=True, density=True)


In [215]:
pred = xgb_r.predict(pd.DataFrame(np.stack(feature_,axis=1)).T)

In [218]:
de_normalizeTargets(pred[0],'train')

7.004355986251742e-07

In [222]:
de_normalizeTargets(target[0],'train')

7.21345845214046e-07